In [2]:
import pandas as pd
import numpy as np
import pickle as pk
import os
import sys

from rdkit import Chem

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from xenonpy.descriptor import Fingerprints
import xenonpy
xenonpy.__version__

from tqdm.autonotebook import tqdm
from radonpy.core import poly, calc, const
from radonpy.ff.gaff2 import GAFF2
from radonpy.ff.descriptor import FF_descriptor
const.print_level = 1


'0.6.5'

### Prepare cyclic SMILES for polymers

The sample codes below demonstrate how to use radonpy to produce SMILES that represents a polymer that undergoes the following two operations:
1. Make N copies of the repeating unit of a polymer and connect them sequentially (head to tail) that results in a long polymer chain
2. Connect the head and tail of the long polymer chain to make a cyclic polymer used to mimic an infinitely long polymer chain

Note that the original polymer SMILES is assumed to have exactly two '*'s that represent the head and tail of a polymer

In [ ]:
N_cyclic = 10 # number of repeating unit for the long polymer chain

smis_single = ['*C(CC)CC*', '*c1ccc(C*)cc1'] # two examples of polymer SMILES

smis_cyclic = [poly.make_cyclicpolymer(x, n=N_cyclic) for x in smis_single]


### Calculate descriptors for polymers and solvents

#### data preparation

Load data: in our data file, SMILES of polymers and solvents are connected with '_' initially. 

In [17]:
data = pd.read_csv('sample_data/data_Chi.csv', index_col=0)
smis_poly = []
smis_solv = []
for smi in data['ps_pair'].values:
    tmp = smi.split('_')
    smis_poly.append(tmp[0])
    smis_solv.append(tmp[1])
    

In [24]:
# extract the unique SMILES of polymers and solvents
uni_poly = np.unique(smis_poly)
uni_solv = np.unique(smis_solv)

print(f'Unique number of polymer SMILES: {uni_poly.shape[0]}')
print(f'Unique number of solvent SMILES: {uni_solv.shape[0]}')


Unique number of polymer SMILES: 28
Unique number of solvent SMILES: 125


To save time, we only calculate the unique SMILES and the assemble the descriptor for each polymer-solvent pair afterward.

In [28]:
# set up a dictionary for descriptor calculation
uni_smis = {'Polymer': uni_poly, 'Solvent': uni_solv}

# set up a dictionary to store all descriptors
desc_data = {}


Note that the final SMILES that we are using for the descriptor calculation contains 'H's explicitly. If the SMILES you have (for both the cyclic polymers and solvents) does not have explicit 'H' representation, please use the follow sample code to modify the SMILES.

In [10]:
smis_noHs = ['CCl', 'ClCCl']
smis_addHs = [Chem.MolToSmiles(Chem.AddHs(Chem.MolFromSmiles(x))) for x in smis_noHs]


#### calculate Force-field descriptors using radonpy

In [6]:
# parameters for force-field descriptors
nk = 20
sigma = 1/nk/2

for key, val in uni_smis.items():
    try:
        ff = GAFF2()
        ff_desc = FF_descriptor(ff, polar=True)
        desc_names = ff_desc.ffkm_desc_names(nk=nk)

        desc = ff_desc.ffkm_mp(list(val), nk=nk, s=sigma, cyclic=0)
            
        desc_data[f'{key}_ff'] = pd.DataFrame(desc, columns=[f'{key}_{x}' for x in desc_names], index=val)
        
        print(datetime.now())
        print(f'{key} done')
        
    except:
        print(f'{key} failed')
        pass
    
print('All done!')


RadonPy warning: Cannot assignment index 9, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 40, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 71, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 102, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 133, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 164, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 195, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 226, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 257, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Cannot assignment index 288, element Co, num. of bonds 2, hybridization SP3D
RadonPy warning: Fail to assign force field. [H]C(=O)c1c([H])c([

#### calculate descriptors from rdkit using xenonpy

In [7]:
%%time

print(datetime.now())
print('Program started...')
for key, val in uni_smis.items():
    mols = [Chem.MolFromSmiles(x) for x in val]
    
    desc_data[f'{key}_rdk'] = Fingerprints(featurizers = 'DescriptorFeature', input_type='mol', on_errors='nan').transform(mols)
    desc_data[f'{key}_rdk']['Ipc'] = np.log(desc_data[f'{key}_rdk']['Ipc'])
    desc_data[f'{key}_rdk'].index = val
    desc_data[f'{key}_rdk'].columns = [f'{key}_{x}' for x in desc_data[f'{key}_rdk'].columns]

    print(datetime.now())
    print(f'{key} done')


2022-10-17 12:18:01.645143
Program started...
2022-10-17 12:24:01.486880
Polymer-ECFP done


/home/liuchang/mambaforge/envs/xepy38/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1748: RuntimeWarning: overflow encountered in reduce
  return asanyarray(a).trace(offset=offset, axis1=axis1, axis2=axis2, dtype=dtype, out=out)
/home/liuchang/mambaforge/envs/xepy38/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1748: RuntimeWarning: invalid value encountered in reduce
  return asanyarray(a).trace(offset=offset, axis1=axis1, axis2=axis2, dtype=dtype, out=out)
/home/liuchang/mambaforge/envs/xepy38/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1748: RuntimeWarning: overflow encountered in reduce
  return asanyarray(a).trace(offset=offset, axis1=axis1, axis2=axis2, dtype=dtype, out=out)
/home/liuchang/mambaforge/envs/xepy38/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1748: RuntimeWarning: invalid value encountered in reduce
  return asanyarray(a).trace(offset=offset, axis1=axis1, axis2=axis2, dtype=dtype, out=out)
/home/liuchang/mambaforge/envs/xepy38/lib/

2022-10-17 12:42:22.919152
Polymer-DescriptorFeature done
2022-10-17 12:42:28.798468
Solvent-ECFP done
2022-10-17 12:42:29.302165
Solvent-DescriptorFeature done
CPU times: user 7min 13s, sys: 24.1 s, total: 7min 37s
Wall time: 24min 27s


/home/liuchang/mambaforge/envs/xepy38/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


#### combine descriptors

In [9]:
# desc_final is in the same format as the descriptor dataframes that are stored in the sample_data folder
desc_final = pd.concat([desc_data['Polymer_ff'].loc[smis_poly,:].reset_index(drop=True),
                       desc_data['Polymer_rdk'].loc[smis_poly,:].reset_index(drop=True),
                       desc_data['Solvent_ff'].loc[smis_solv,:].reset_index(drop=True),
                       desc_data['Solvent_rdk'].loc[smis_solv,:].reset_index(drop=True)], axis=1)
